In [1]:
import numpy as np
from time import time
import matplotlib.pyplot as plt
import rustworkx as rx

from qiskit_nature.problems.second_quantization.lattice import Lattice
import sys
import os
import time

from heisenberg_model import HeisenbergModel
from qiskit_nature.mappers.second_quantization import LogarithmicMapper
from qiskit.algorithms import NumPyEigensolver
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit import Aer, transpile
from qiskit.tools.visualization import plot_histogram

import dask
from distributed import Client
import dask.array as da

from kagome_custom import custom_VQE

from qiskit_aer import AerSimulator



In [2]:
#path_to_json = '../dask_cluster_ft3/scheduler_info.json' #CAMBIAR
#client = Client(scheduler_file = path_to_json)
client = Client(n_workers=12)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 12
Total threads: 12,Total memory: 15.91 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64493,Workers: 12
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.91 GiB
Comm: tcp://127.0.0.1:64559,Total threads: 1
Dashboard: http://127.0.0.1:64567/status,Memory: 1.33 GiB
Nanny: tcp://127.0.0.1:64496,


In [3]:
def get_energy(vqe,circuit):
        
        results = vqe.aer_sim.run(circuit,shots=vqe.shots).result()
        counts = results.get_counts()
        
        energy = 0
        for edge in vqe.edge_list:
            for num in counts:
                factor = 1
                for n in range(len(num)):
                    n_corr = len(num) - n - 1
                    digit = int(num[n_corr])
                    if n == edge[0] or n == edge[1]:
                        if digit == 1:
                            factor *= -1
                energy += factor*counts[num]

        energy = energy/vqe.shots
        return energy
    
def energy_estimation(vqe,param):
        transpiled_qc_meas = [None, None, None]
        for j in range(3):
            transpiled_qc_meas[j] = vqe.qc_meas[j].assign_parameters(param)
            transpiled_qc_meas[j] = transpile(transpiled_qc_meas[j],vqe.aer_sim)
        
  
        energies = [None,None,None]
        for axis in range(3):
            energies[axis] = get_energy(vqe,transpiled_qc_meas[axis])

        energy_tot = sum(energies)
        return energy_tot

In [4]:
# Guardar energías anterior iteración

@dask.delayed
def binomial(vqe,npop,population,num_p,cr,f):
    
        x = population[npop]
               
        valid_index = [ind for ind in range(num_p) if ind != npop]
    
        cross_index = np.random.choice(valid_index, 3, replace = False)

        a = population[cross_index[0]]
        #a = best_param
        
        b = population[cross_index[1]]
        c = population[cross_index[2]]
        r = np.random.randint(0,vqe.num_param)
        y = np.array([])

        for yi in range(vqe.num_param):
            ri = np.random.uniform(0,1)
            if ri < cr or yi == r:
                y = np.append(y,(a[yi] + f * (b[yi] - c[yi]))%(np.pi))
            else :
                y = np.append(y,x[yi])
        
        ener_y = energy_estimation(vqe,y)
        ener_x = energy_estimation(vqe,x)
        
        best = []
        if ener_y < ener_x:
            best = [y,ener_y]
            
        else :
            best = [x,ener_x] 

        return best
      

        
@dask.delayed
def exponential(vqe,npop,population,num_p,cr,f):
    
        x = population[npop]
               
        valid_index = [ind for ind in range(num_p) if ind != npop]
    
        cross_index = np.random.choice(valid_index, 3, replace = False)

        a = population[cross_index[0]]
        #a = best_param
        b = population[cross_index[1]]
        c = population[cross_index[2]]
        r = np.random.randint(0,vqe.num_param)
        y = [0]*vqe.num_param
        
        for yi in range(vqe.num_param):
            y[yi]=x[yi]
        exp_index = np.random.choice(range(0,vqe.num_param-1), 2, replace = False)
        exp_index = np.sort(exp_index)
        
        for yi in range(exp_index[0],exp_index[1]+1):
                y[yi] = (a[yi] + f * (b[yi] - c[yi]))%(np.pi)
                
#         print(exp_index)
#         print(x)
#         print(y)        
            
        
        ener_y = energy_estimation(vqe,y)
        ener_x = vqe.population_energies[npop]
        #print(ener_x,ener_y)
        #ener_x = energy_estimation(vqe,x)
        #print(str(npop)+': '+str(ener_y)+' '+str(ener_x))
  
        
        
        
        best = []
        if ener_y < ener_x:
            best = [y,ener_y]
            
        else :
            best = [x,ener_x] 

        return best
        
    

In [5]:
def diff_ev(vqe, init_param, itmax=10000, num_p=20, cr=0.9, f=0.8):
        
        #tol=1/np.sqrt(vqe.shots)
        old_pop = []
        for npop in range(num_p):
            old_pop.append(init_param + (-1+2*np.random.rand(vqe.num_param))*1.e-2)
        
        old_energy = energy_estimation(vqe,init_param)
        info = [old_energy]
        
        print('Energy Z-ground state: '+str(old_energy))
        print()
        
        population = old_pop
        vqe.population_energies = []
        for npop in range(num_p):
            vqe.population_energies.append(0)
        
        for npop in range(num_p):
            vqe.population_energies[npop] = energy_estimation(vqe,population[npop])
        
        best_energy = old_energy
        
        file_name = 'results83.txt'
        file=open(file_name, 'w')
        file.close()
        
        for it in range(itmax):
            print('-----Iteration '+str(it+1)+'-----')
            
            cond_list = []
            for npop in range(num_p):
                cond_list.append(None)
    
            
            for npop in range(num_p):
                cond_list[npop] = exponential(vqe,npop,population,num_p,cr,f) #####
 
            cond_list = dask.compute(cond_list)
            cond_list = cond_list[0]
            
            
            for npop in range(num_p):
                population[npop] = cond_list[npop][0]
                vqe.population_energies[npop] = cond_list[npop][1]
        
            
            for indv in range(len(population)):
                indv_energy = vqe.population_energies[indv]
                
                if indv_energy < best_energy:
                    best_energy = indv_energy
            

            print(best_energy)

            
            
            file = open(file_name,'a')
            file.writelines(str(best_energy)+'\n')
            file.close()
                    
#  Checkear que esté todo bien!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#             if np.std(population_energies) < tol * np.abs(np.mean(population_energies)):
#                 for data_point in info[-(it+1)%1000:]:
#                     file = open(file_name,'a')
#                     file.writelines(str(data_point)+'\n')
#                     file.close()
#                 break     
                   
  
        return population

In [6]:
if __name__ == '__main__':
    t = 1.0
    edge_list = [
        (1, 2, t),
        (2, 3, t),
        (3, 5, t),
        (5, 8, t),
        (8, 11, t),
        (11, 0, t),
        (0, 6, t),
        (6, 9, t),
        (9, 10, t),
        (10, 7, t),
        (7, 4, t),
        (4, 1, t),
        (4, 2, t),
        (2, 5, t),
        (5, 11, t),
        (11, 6, t),
        (6, 10, t),
        (10, 4, t),
    ]


    num_qubits = 12
    layers = 6
    shots = 1000


    
    kagome = custom_VQE(edge_list, num_qubits, layers, shots)
    kagome.get_backend()

    

    


In [7]:
init_param = np.pi*(2*np.random.rand(kagome.num_param)-1)*0
values = diff_ev(kagome,init_param)

Energy Z-ground state: -6.156

-----Iteration 1-----
-6.5600000000000005
-----Iteration 2-----
-6.676
-----Iteration 3-----
-6.676
-----Iteration 4-----
-6.676
-----Iteration 5-----
-6.676
-----Iteration 6-----
-6.676
-----Iteration 7-----
-6.676
-----Iteration 8-----
-7.696
-----Iteration 9-----
-7.696
-----Iteration 10-----
-8.404
-----Iteration 11-----
-8.404
-----Iteration 12-----


KeyboardInterrupt: 